FELIX ZEBAOTH


In [1]:
!pip install -q pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [3]:
spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName("test") \
    .getOrCreate()


In [5]:
from google.colab import files
uploaded = files.upload()


Saving book.txt to book.txt
Saving fakefriends-header.csv to fakefriends-header.csv
Saving fakefriends.csv to fakefriends.csv
Saving sephora_website_dataset.csv to sephora_website_dataset.csv
Saving worldcities.csv to worldcities.csv


In [6]:
df1 = spark.read.option("delimiter", ",").csv("worldcities.csv")
df1.show()
df1.printSchema()


+------------+------------+--------+--------+-------------+----+----+--------------------+-------+----------+----------+
|         _c0|         _c1|     _c2|     _c3|          _c4| _c5| _c6|                 _c7|    _c8|       _c9|      _c10|
+------------+------------+--------+--------+-------------+----+----+--------------------+-------+----------+----------+
|        city|  city_ascii|     lat|     lng|      country|iso2|iso3|          admin_name|capital|population|        id|
|       Tokyo|       Tokyo| 35.6897|139.6922|        Japan|  JP| JPN|               Tōkyō|primary|  37977000|1392685764|
|     Jakarta|     Jakarta| -6.2146|106.8451|    Indonesia|  ID| IDN|             Jakarta|primary|  34540000|1360771077|
|       Delhi|       Delhi| 28.6600| 77.2300|        India|  IN| IND|               Delhi|  admin|  29617000|1356872604|
|      Mumbai|      Mumbai| 18.9667| 72.8333|        India|  IN| IND|         Mahārāshtra|  admin|  23355000|1356226629|
|      Manila|      Manila| 14.6

In [9]:

manualSchema = StructType([
               StructField("city", StringType(), True),
               StructField("city_ascii", StringType(), True),
               StructField("lat", FloatType(), True),
               StructField("lng", FloatType(), True),
               StructField("country", StringType(), True),
               StructField("iso2", StringType(), True),
               StructField("iso3", StringType(), True),
               StructField("admin_name", StringType(), True),
               StructField("capital", StringType(), True),
               StructField("population", IntegerType(), True),
               StructField("id", StringType(), True)
])


In [10]:
df_city = spark.read.format("csv").option("header", "true").schema(manualSchema).load("worldcities.csv")
df_city.show(10)
df_city.printSchema()


+-----------+-----------+--------+--------+------------+----+----+----------------+-------+----------+----------+
|       city| city_ascii|     lat|     lng|     country|iso2|iso3|      admin_name|capital|population|        id|
+-----------+-----------+--------+--------+------------+----+----+----------------+-------+----------+----------+
|      Tokyo|      Tokyo| 35.6897|139.6922|       Japan|  JP| JPN|           Tōkyō|primary|  37977000|1392685764|
|    Jakarta|    Jakarta| -6.2146|106.8451|   Indonesia|  ID| IDN|         Jakarta|primary|  34540000|1360771077|
|      Delhi|      Delhi|   28.66|   77.23|       India|  IN| IND|           Delhi|  admin|  29617000|1356872604|
|     Mumbai|     Mumbai| 18.9667| 72.8333|       India|  IN| IND|     Mahārāshtra|  admin|  23355000|1356226629|
|     Manila|     Manila|    14.6|120.9833| Philippines|  PH| PHL|          Manila|primary|  23088000|1608618140|
|   Shanghai|   Shanghai| 31.1667|121.4667|       China|  CN| CHN|        Shanghai|  adm

In [11]:
df_city.createOrReplaceTempView("worldcities")

results = spark.sql("SELECT country, SUM(population) as total_population FROM worldcities GROUP BY country ORDER BY total_population DESC")

results.show(5)

+-------------+----------------+
|      country|total_population|
+-------------+----------------+
|        China|      1446111841|
|United States|       401453709|
|        India|       270309635|
|       Brazil|       198554881|
|        Japan|       138634635|
+-------------+----------------+
only showing top 5 rows



In [12]:

results.write.mode('overwrite').option('compression', 'snappy').save('total_population_worldcities')